In [1]:
import gzip
import pandas as pd
import shutil
import os
from google.colab import files
import numpy as np

In [2]:
!git clone https://github.com/eliasezar27/PRESANA.git

Cloning into 'PRESANA'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 118 (delta 23), reused 42 (delta 19), pack-reused 72
Receiving objects: 100% (118/118), 1.23 GiB | 40.54 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Checking out files: 100% (48/48), done.


In [3]:
base_dir = '/content/PRESANA/hawaii_airBnbGroup'
gz_files_dir = os.path.join(base_dir, 'hawaii_airbnbopendataset')

In [4]:
def gzipTocsv(gz_file, csv_file):
    with gzip.open(gz_file, 'rb') as f_in:
        with open(csv_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [5]:
for i in os.listdir(gz_files_dir):
    gz_file = os.path.join(gz_files_dir, i)
    csv_file = os.path.join(gz_files_dir, i[:-3])
    gzipTocsv(gz_file, csv_file)

In [7]:
csvList = os.listdir(gz_files_dir)
csvList.sort()

### **Calendar Data Merging**

In [ ]:
calendar_df = pd.DataFrame([])
for j in csvList:
    if "calendar" in j and not "gz" in j:
        indiv_calendar = pd.read_csv(os.path.join(gz_files_dir, j))
        calendar_df = pd.concat([calendar_df, indiv_calendar])
        calendar_df = calendar_df.drop_duplicates(subset=['listing_id', 'date'], keep='last')

In [ ]:
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
calendar_df['date'].min(), calendar_df['date'].max()

(Timestamp('2020-12-19 00:00:00'), Timestamp('2022-10-16 00:00:00'))

In [ ]:
calendar_df = calendar_df.drop(columns=['price', 'minimum_nights', 'maximum_nights'])
calendar_df['adjusted_price'] = calendar_df['adjusted_price'].map(lambda x: str(x).strip('$').replace(',',''))
calendar_df = calendar_df.sort_values(['date','listing_id'])
calendar_df = calendar_df[calendar_df['date'] < '2021-12-04']
calendar_df = calendar_df.reset_index(drop=True)

In [ ]:
print("Data is from", calendar_df['date'].min(), " to ", calendar_df['date'].max())
calendar_df.info()

Data is from 2020-12-19 00:00:00  to  2021-12-03 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9300867 entries, 0 to 9300866
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   listing_id      int64         
 1   date            datetime64[ns]
 2   available       object        
 3   adjusted_price  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 283.8+ MB


In [ ]:
prtn1, prtn2, prtn3, prtn4 = np.array_split(calendar_df, 4)

calendar_df1 = pd.DataFrame(prtn1)
calendar_df2 = pd.DataFrame(prtn2)
calendar_df3 = pd.DataFrame(prtn3)
calendar_df4 = pd.DataFrame(prtn4)

calendar_df1.to_csv(os.path.join(base_dir, 'calendar_main_part1.csv'), index=False)
calendar_df2.to_csv(os.path.join(base_dir, 'calendar_main_part2.csv'), index=False)
calendar_df3.to_csv(os.path.join(base_dir, 'calendar_main_part3.csv'), index=False)
calendar_df4.to_csv(os.path.join(base_dir, 'calendar_main_part4.csv'), index=False)

files.download(os.path.join(base_dir, 'calendar_main_part1.csv'))
files.download(os.path.join(base_dir, 'calendar_main_part2.csv'))
files.download(os.path.join(base_dir, 'calendar_main_part3.csv'))
files.download(os.path.join(base_dir, 'calendar_main_part4.csv'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Listings Data Merging**

In [8]:
listings_df = pd.DataFrame([])
for j in csvList:
    if "listings" in j and not "gz" in j:
        indiv_listings = pd.read_csv(os.path.join(gz_files_dir, j))
        listings_df = pd.concat([listings_df, indiv_listings])
        listings_df = listings_df.drop_duplicates(subset=['id'], keep='last')

In [9]:
listings_df['price'] = listings_df['price'].map(lambda x: str(x).strip('$').replace(',',''))
listings_df = listings_df.sort_values(['id'])
listings_df = listings_df.reset_index(drop=True)
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31450 entries, 0 to 31449
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            31450 non-null  int64  
 1   listing_url                                   31450 non-null  object 
 2   scrape_id                                     31450 non-null  int64  
 3   last_scraped                                  31450 non-null  object 
 4   name                                          31450 non-null  object 
 5   description                                   31077 non-null  object 
 6   neighborhood_overview                         17223 non-null  object 
 7   picture_url                                   31450 non-null  object 
 8   host_id                                       31450 non-null  int64  
 9   host_url                                      31450 non-null 

In [10]:
listings_df.to_csv(os.path.join(base_dir, 'listings_main.csv'), index=False)
files.download(os.path.join(base_dir, 'listings_main.csv'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Reviews data selection**

In [ ]:
reviews_df = pd.read_csv('/content/PRESANA/hawaii_airBnbGroup/hawaii_airbnbopendataset/reviews_2021-10-11.csv')
reviews_df['date'] = pd.to_datetime(reviews_df['date'])
reviews_df = reviews_df.sort_values(['date', 'id'])
reviews_df = reviews_df.reset_index(drop=True)
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676791 entries, 0 to 676790
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   listing_id     676791 non-null  int64         
 1   id             676791 non-null  int64         
 2   date           676791 non-null  datetime64[ns]
 3   reviewer_id    676791 non-null  int64         
 4   reviewer_name  676791 non-null  object        
 5   comments       676431 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 31.0+ MB


In [ ]:
prtn1, prtn2, prtn3, prtn4 = np.array_split(reviews_df, 4)

reviews_df1 = pd.DataFrame(prtn1)
reviews_df2 = pd.DataFrame(prtn2)
reviews_df3 = pd.DataFrame(prtn3)
reviews_df4 = pd.DataFrame(prtn4)

reviews_df1.to_csv(os.path.join(base_dir, 'reviews_main_part1.csv'), index=False)
reviews_df2.to_csv(os.path.join(base_dir, 'reviews_main_part2.csv'), index=False)
reviews_df3.to_csv(os.path.join(base_dir, 'reviews_main_part3.csv'), index=False)
reviews_df4.to_csv(os.path.join(base_dir, 'reviews_main_part4.csv'), index=False)

files.download(os.path.join(base_dir, 'reviews_main_part1.csv'))
files.download(os.path.join(base_dir, 'reviews_main_part2.csv'))
files.download(os.path.join(base_dir, 'reviews_main_part3.csv'))
files.download(os.path.join(base_dir, 'reviews_main_part4.csv'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>